# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [132]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [133]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [134]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [135]:
grouped =data.groupby(["CustomerID", "ProductName"])["Quantity"].sum().to_frame()
grouped

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [136]:
df2 = pd.pivot_table(df,index=["ProductName"],columns =["CustomerID"],values="Quantity", fill_value = 0)
df2

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [220]:
from scipy.spatial.distance import pdist, squareform

distances = pdist(df2.T, 'euclidean')
distances

array([ 11.91637529,  10.48808848,  11.22497216, ..., 304.13812651,
       305.16389039, 303.10889132])

In [221]:
distances = pd.DataFrame(1/(1 + squareform(pdist(df2.T, 'euclidean'))), 
                         index=df2.columns, columns=df2.columns)

distances

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [230]:
similarities = distances[33].sort_values(ascending=False)[1:]
top5 = similarities[0:5].index

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [233]:
df3 = grouped.loc[top5]
df3.reset_index()


,CustomerID,ProductName,Quantity
0,264,Apricots - Halves,1
1,264,Apricots Fresh,1
2,264,Bacardi Breezer - Tropical,1
3,264,Bagel - Plain,1
4,264,Banana - Leaves,1
...,...,...,...
265,3535,Tray - 16in Rnd Blk,1
266,3535,Wanton Wrap,1
267,3535,Wine - Blue Nun Qualitatswein,1
268,3535,"Wine - Magnotta, Merlot Sr Vqa",1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [141]:
df4 =df3.groupby(['ProductName'])["Quantity"].sum().to_frame()
df4.sort_values(["Quantity"],ascending=False)

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3
...,...
Hinge W Undercut,1
Ice Cream Bar - Hageen Daz To,1
Jagermeister,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [153]:
customer33 =df2[33].to_frame()
customer33


,33
ProductName,
Anchovy Paste - 56 G Tube,0
"Appetizer - Mini Egg Roll, Shrimp",0
Appetizer - Mushroom Tart,0
Appetizer - Sausage Rolls,0
Apricots - Dried,1
...,...
Yeast Dry - Fermipan,0
Yoghurt Tubes,0
"Yogurt - Blueberry, 175 Gr",0


In [237]:
merge= df2.merge(df4 , on = "ProductName" )
limpio  =merge.copy()
limpio=limpio[[33, "Quantity"]]
limpio =limpio[33]= limpio[limpio[33] ==0]
limpio =limpio.dropna()
limpio.sort_values(["Quantity"],ascending=False)[0:5]

,33,Quantity
ProductName,,
Beef Wellington,0,3
Wine - Blue Nun Qualitatswein,0,3
Wine - Ej Gallo Sierra Valley,0,3
Soup - Campbells Bean Medley,0,3
Butter - Unsalted,0,3


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [209]:
customers =data.CustomerID.unique()
dictionary= {}

        



In [258]:
for c in customers:
    similarities = distances[c].sort_values(ascending=False)[1:]
    top5 = similarities[0:5].index
    df3 = grouped.loc[top5]
    df3.reset_index()
    df4 =df3.groupby(['ProductName'])["Quantity"].sum().to_frame()
    df4.sort_values(["Quantity"],ascending=False)
    customer33 =df2[c].to_frame()
    merge= df2.merge(df4 , on = "ProductName" )
    limpio  =merge.copy()
    limpio=limpio[[c, "Quantity"]]
    limpio =limpio[c]= limpio[limpio[c] ==0]
    limpio =limpio.dropna()
    x = limpio.sort_values(["Quantity"],ascending=False)[0:5]
    dictionary[c] = x

In [212]:
##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [261]:
# es un jaleo me he creado un diccionario maestro pero me va a salir cada dataframe separado
dictionary

{'a':                                3472  Quantity
 ProductName                                  
 Soup - Campbells Bean Medley      0         3
 Wine - Blue Nun Qualitatswein     0         3
 Pepper - Black, Whole             0         3
 Wine - Ej Gallo Sierra Valley     0         3
 Butter - Unsalted                 0         3,
 61288:                            61288  Quantity
 ProductName                               
 Chicken - Soup Base            0        12
 Jagermeister                   0        12
 Lime Cordial - Roses           0        10
 Macaroons - Two Bite Choc      0         9
 Water - Mineral, Natural       0         8,
 77352:                           77352  Quantity
 ProductName                              
 Wine - Valpolicella Masi      0        18
 Tahini Paste                  0        18
 Tuna - Salad Premix           0        17
 Chicken - Soup Base           0        16
 Isomalt                       0        15,
 40094:                                 

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

{'a':                                3472  Quantity
 ProductName                                  
 Soup - Campbells Bean Medley      0         3
 Wine - Blue Nun Qualitatswein     0         3
 Pepper - Black, Whole             0         3
 Wine - Ej Gallo Sierra Valley     0         3
 Butter - Unsalted                 0         3}